In [0]:
%sql
DROP TABLE IF EXISTS `databricks-uc-catalog`.`first_schema`.`source_data_ctc`;
DROP VIEW IF EXISTS `databricks-uc-catalog`.`first_schema`.`source_data_ctc_view`;
DROP VIEW IF EXISTS `databricks-uc-catalog`.`first_schema`.`additional_data_view`;

Created a table in unity catalog and stored it in abfss

In [0]:
CREATE OR REPLACE TABLE `databricks-uc-catalog`.first_schema.source_data_ctc(
    id INT,
    address STRING,
    current BOOLEAN,
    effective_date string,
    end_date string
    ) USING DELTA
    LOCATION 'abfss://metastore@databricksugdl.dfs.core.windows.net/data_capture'
    TBLPROPERTIES (delta.enableChangeDataFeed = true);



Inserted some code into the table


In [0]:
INSERT INTO `databricks-uc-catalog`.first_schema.source_data_ctc (id, address, current, effective_date, end_date)
VALUES
    (1, '123 Main St', true, '2025-08-01', '2025-08-31'),
    (2, '456 Oak Ave', false, '2025-07-15', '2025-08-14'),
    (3, '789 Pine Rd', true, '2025-08-10', NULL),
    (4, '321 Maple Ln', false, '2025-06-01', '2025-07-01');

Created a view and added some data one is new data and another one is updated data

In [0]:
CREATE OR REPLACE TEMPORARY VIEW source_data_ctc_view AS
SELECT 1 id, "An updated address" as address, true as current, '2025-09-12' as effective_date, NULL as end_date
UNION ALL
SELECT 5, 'New added data', true, '2025-09-12', NULL as end_date

Merged the new to old data

In [0]:
MERGE INTO `databricks-uc-catalog`.first_schema.source_data_ctc AS original
USING source_data_ctc_view AS source
ON source.id = original.id
WHEN MATCHED THEN
  UPDATE SET
    original.address = source.address,
    original.current = source.current,
    original.effective_date = source.effective_date,
    original.end_date = source.end_date
WHEN NOT MATCHED THEN
  INSERT (id, address, current, effective_date, end_date)
  VALUES (source.id, source.address, source.current, source.effective_date, source.end_date);

In [0]:
SELECT * FROM `databricks-uc-catalog`.first_schema.source_data_ctc

In [0]:
%python 
df=spark.read.parquet("abfss://metastore@databricksugdl.dfs.core.windows.net/data_capture/_change_data/cdc-00000-0f3bc8e4-a75c-4355-ab77-0a558ad9c7b5.c000.snappy.parquet")
display(df)

In [0]:
describe history `databricks-uc-catalog`.`first_schema`.`source_data_ctc`

Checking on the version from 1 and 2 inclusive

In [0]:
SELECT * FROM table_changes('`databricks-uc-catalog`.first_schema.source_data_ctc',1,2)

In [0]:
SELECT * FROM table_changes('`databricks-uc-catalog`.first_schema.source_data_ctc',3)

In [0]:
SELECT * 
FROM table_changes('`databricks-uc-catalog`.first_schema.source_data_ctc',1,2)  
WHERE _change_type='update_preimage'

In [0]:
select * from `databricks-uc-catalog`.first_schema.source_data_ctc

In [0]:
INSERT INTO `databricks-uc-catalog`.first_schema.source_data_ctc (id, address, current, effective_date, end_date)
VALUES
    (6, '1234 Main Street', true, '2025-08-01', '2025-08-31')